In [ ]:
from PIL import Image
import keras
from keras_preprocessing.image import ImageDataGenerator,load_img,img_to_array
from keras.applications.vgg19 import VGG19, preprocess_input, decode_predictions
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf


In [ ]:
!pip install tensorflow_model_optimization

In [ ]:
import tensorflow_model_optimization as tmot


In [ ]:
tmot

In [ ]:
train_directory = (r'C:\Users\vrush\Downloads\plant\image data\train')
val_directory = (r'C:\Users\vrush\Downloads\plant\image data\validation')

In [ ]:
train_datagen= ImageDataGenerator(zoom_range=0.5, shear_range=0.3, horizontal_flip=True, preprocessing_function=preprocess_input)
val_datagen= ImageDataGenerator(preprocessing_function=preprocess_input)

In [ ]:
def train_preprocess(path_preprocess):
  train= train_datagen.flow_from_directory(directory=path_preprocess, target_size=(256,256), batch_size=32)
  return train

In [ ]:
def validation_preprocess(path_preprocess):
  val= val_datagen.flow_from_directory(directory= path_preprocess, target_size=(256,256), batch_size=32)
  return val

In [ ]:
train_preprocess(train_directory)

In [ ]:
train =train_preprocess(train_directory)

In [ ]:
val =validation_preprocess(val_directory)


CHECKING PREPROCESSING:

In [ ]:
t_img,label=train.next()

In [ ]:
def plotImage(img_arr,label):
  for im,l in zip(img_arr,label):
    plt.figure(figsize=(5,5))
    plt.imshow(im)
    plt.show()

In [ ]:
plotImage(t_img[:3],label[:3])

BUILDING MODEL:

In [ ]:
from keras.layers import Dense, Flatten
from keras.models import Model
import keras
from keras.applications.vgg19 import VGG19

In [ ]:
base_model = VGG19(input_shape=(256,256,3), include_top=False)

In [ ]:
for layer in base_model.layers:
  layer.trainable=False

In [ ]:
base_model.summary()

In [ ]:
X = Flatten()(base_model.output)
X = Dense(units=13, activation='softmax')(X)

model=Model(base_model.input, X)

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

early stopping and model checkpoint

In [ ]:
from keras.callbacks import ModelCheckpoint,EarlyStopping
#early stopping
es=EarlyStopping(monitor='val_accuracy', min_delta=0.01,patience=3,verbose=1)

#model check point
mc=ModelCheckpoint(filepath="best_model_new.h5",monitor='val_accuracy', min_delta=0.01,patience=3,verbose=1,save_best_only=True)


cb=[es,mc]

In [ ]:
his=model.fit_generator(train, 
                        steps_per_epoch=16,
                        epochs=50, 
                        verbose=1, 
                        callbacks=cb, 
                        validation_data=val, 
                        validation_steps=16 )

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
h=his.history
h.keys()

In [ ]:
plt.plot(h['accuracy'])
plt.plot(h['val_accuracy'], c='red')
plt.title('acc vs v-acc')
plt.show()

In [ ]:
plt.plot(h['loss'])
plt.plot(h['val_loss'], c='red')
plt.title('loss vs v-loss')
plt.show()

LOADING BEST MODEL

In [ ]:
from keras.models import load_model
model=load_model(r"C:\Users\vrush\best_model_new.h5")

In [ ]:
from tensorflow.python import keras
import tensorflow


converter= tensorflow.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()


In [ ]:

converter.optimizations = [tf.lite.Optimize.DEFAULT]


tflite_quant_model = converter.convert()

In [ ]:
len(tflite_quant_model)

In [ ]:
with open("converted_model_newer.tflite","wb") as f:
    f.write(tflite_model)

with open("converted_quant_model_newer.tflite", "wb") as f:
    f.write(tflite_quant_model)
    

In [ ]:
import tensorflow_model_optimization as tfmot
quantize_model = tfmot.quantization.keras.quantize_model
q_aware_model = quantize_model(model)

q_aware_model.compile(optimizer='adam',loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])
q_aware_model.summary()

In [ ]:
q_aware_model.fit_generator(train, 
                        steps_per_epoch=16,
                        epochs=10, 
                        verbose=1, 
                        callbacks=cb, 
                        validation_data=val, 
                        validation_steps=16 )

In [ ]:
converter= tf.lite.TFLiteConverter.from_keras_model(q_aware_model)
converter.optimization = [tf.lite.Optimize.DEFAULT]
tflite_qaware_model = converter.convert()

In [ ]:
with open("tflite_qaware_model.tflite","wb") as f:
    f.write(tflite_qaware_model)

In [ ]:
acc=q_aware_model.evaluate_generator(val)[1]
print(f"The accuracy of your model is ={acc*100}%")

In [ ]:
! pip install -q tensorflow-model-optimization


In [ ]:
import tempfile
import os

import tensorflow as tf
import numpy as np

from tensorflow import keras

%load_ext tensorboard

In [ ]:
import tensorflow_model_optimization as tfmot

TESTING

In [ ]:
ref =dict(zip(list(train.class_indices.values()), list(train.class_indices.keys())))
dict(zip(list(train.class_indices.values()), list(train.class_indices.keys())))

In [ ]:
path=r"C:\Users\vrush\Downloads\plant\image data\test\orange\haunglongbing (citrus greening)\00c44fe5-1ee3-481a-a309-7a9784a4e8c9___UF.Citrus_HLB_Lab 0504.JPG"
# prediction(path)


In [ ]:
img = load_img(path, target_size=(256,256))
i=img_to_array(img)
im=preprocess_input(i)

img=np.expand_dims(im,axis=0)

pred= np.argmax(model.predict(img))
print(ref[pred])

In [ ]:
from pathlib import Path
def predictfinal(path):
  img = load_img(path, target_size=(256,256))
  i=img_to_array(img)
  im=preprocess_input(i)

  img=np.expand_dims(im,axis=0)

  pred= np.argmax(model.predict(img))
  print(ref[pred])
  data_folder = Path("C:/Users/vrush/Downloads/plant/image data/train/")
  dir_path = data_folder / ref[pred]
  data_folder_n = Path("C:/Users/vrush/Downloads/plant/image data/validation")
  dir_pathone = data_folder_n / ref[pred]                   
  print(dir_path)
 
  train_inner = train_preprocess(dir_path)
  val_inner =validation_preprocess(dir_pathone)
  class_number = os.listdir(dir_path)
  X_inner = Flatten()(base_model.output)
  X_inner= Dense(units=len(class_number), activation='softmax')(X_inner)

  model_inner=Model(base_model.input, X_inner)
  model_inner.compile(optimizer='adam',loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])
  #early stopping
  es_inner=EarlyStopping(monitor='val_accuracy', min_delta=0.01,patience=3,verbose=1)

  #model check point
  mc_inner=ModelCheckpoint(filepath="best_model_inner.h5",monitor='val_accuracy', min_delta=0.01,patience=3,verbose=1,save_best_only=True)


  cb_inner=[es_inner,mc_inner]
  his_inner=model_inner.fit_generator(train_inner, 
                        steps_per_epoch=16,
                        epochs=1, 
                        verbose=1, 
                        callbacks=cb_inner, 
                        validation_data=val_inner, 
                        validation_steps=2 )
  model_inner=load_model(r"C:\Users\vrush\best_model_inner.h5")
  modelnew_inner = keras.models.load_model(model_inner)
  converter= tensorflow.lite.TFLiteConverter.from_keras_model(modelnew_inner)
  tflite_model_inner = converter.convert()
  open("converted_model_inner.tflite","wb").write(tflite_model_inner)
  ref_inner =dict(zip(list(train_inner.class_indices.values()), list(train_inner.class_indices.keys())))
  print(ref_inner)
  # img = load_img(path, target_size=(256,256))
  # i=img_to_array(img)
  # im=preprocess_input(i)

  # img=np.expand_dims(im,axis=0)

  pred= np.argmax(model_inner.predict(img))
  print(f"the image belongs to {ref_inner[pred]}")

  


In [ ]:
predictfinal(path)